In [6]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf

# 加载数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
# 设置训练参数
learning_rate = 0.001 #学习率
num_steps = 200 #训练步数
batch_size = 128 #每次训练的数据集大小
display_step = 10
# 神经网络的参数
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST 数据集共有10个数字(0-9 )
dropout = 0.25 # 设置Dropout率

# 输入
X = tf.placeholder(tf.float32, [None, num_input]) #X为28*28
Y = tf.placeholder(tf.float32, [None, num_classes]) #Y为10
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)


# 建立卷基层，输入的参数是input,weights,b
def conv2d(x, W, b, strides=1):
    #‘SAME’后卷积前后尺度不变
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    #添加bias
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # 池化层，kernel尺寸2*2，滑动2*2
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')




Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [7]:
# 建立模型
def conv_net(x, weights, biases, dropout):
    #输入数据
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # 卷积层1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # 池化层1
    conv1 = maxpool2d(conv1, k=2)

    # 卷积层2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # 池化层2(down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # 全连接层
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # 设置Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # 输出
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


In [8]:
# 初始化weights
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)


In [9]:
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.8})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))


Step 1, Minibatch Loss= 97033.1328, Training Accuracy= 0.094
Step 10, Minibatch Loss= 29547.2402, Training Accuracy= 0.203
Step 20, Minibatch Loss= 12557.4336, Training Accuracy= 0.453
Step 30, Minibatch Loss= 6575.0674, Training Accuracy= 0.578
Step 40, Minibatch Loss= 4490.9438, Training Accuracy= 0.688
Step 50, Minibatch Loss= 4598.7529, Training Accuracy= 0.703
Step 60, Minibatch Loss= 3108.2166, Training Accuracy= 0.805
Step 70, Minibatch Loss= 3655.3667, Training Accuracy= 0.836
Step 80, Minibatch Loss= 2946.2429, Training Accuracy= 0.836
Step 90, Minibatch Loss= 2455.6982, Training Accuracy= 0.820
Step 100, Minibatch Loss= 1929.3008, Training Accuracy= 0.906
Step 110, Minibatch Loss= 3338.7092, Training Accuracy= 0.852
Step 120, Minibatch Loss= 1774.7366, Training Accuracy= 0.883
Step 130, Minibatch Loss= 2127.3201, Training Accuracy= 0.875
Step 140, Minibatch Loss= 1328.6914, Training Accuracy= 0.898
Step 150, Minibatch Loss= 972.1561, Training Accuracy= 0.922
Step 160, Minibat